# Assembling our quarterback dataset
Along with the EDA, we also do the data wrangling to put together the dataset we'll use for our quarterback model.<br>

In [254]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import nfl_data_py as nfl
from functions import get_current_weekday, calculate_nfl_week, get_next_sunday, get_current_year

In [255]:
import re

In [256]:
day = get_current_weekday()

In [257]:
date_string = get_next_sunday(day)

In [258]:
week = calculate_nfl_week(date_string)

In [259]:
season = get_current_year()

In [260]:
import sqlite3

In [261]:
# Connect to the SQLite database
#These are the FD and DK player lists for the current week
conn = sqlite3.connect('nfl_dfs.db')

query_fd = "SELECT * FROM fd_table_" + str(week) + "_" + str(season)[2:]
query_dk = "SELECT * FROM dk_table_" + str(week) + "_" + str(season)[2:]

fanduel_df = pd.read_sql_query(query_fd, conn)
draftkings_df = pd.read_sql_query(query_dk, conn)

# Close the database connection
conn.close()

In [262]:
#The weekly and play-by-play data through the previous week
#We went into R Studio to get these and save them as CSVs that we can use here
weekly_df = pd.read_csv('weekly_data_' + str(season) + '_' + str(week) + '.csv')
pbp_df = pd.read_csv('pbp_data_' + str(season) + '_' + str(week) + '.csv', low_memory = False)

In [263]:
weekly_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,1,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,10.38,10.38
1,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,2,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,9.40,9.40
2,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,3,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,14.74,14.74
3,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,4,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,25.40,25.40
4,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,5,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,19.74,19.74


In [264]:
weekly_df.tail()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
12513,00-0039920,M.Corley,Malachi Corley,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,2,REG,...,0,-0.475780,0,0.000000,0.034483,-0.006579,0.047119,0,0.4,1.4
12514,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,1,REG,...,0,-0.088616,0,-1.250000,0.032258,-0.020942,0.033727,0,1.8,2.8
12515,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,2,REG,...,0,0.238474,0,-1.166667,0.047619,-0.028037,0.051802,0,1.7,2.7
12516,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,3,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.8,0.8
12517,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,4,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,5.0,5.0


In [265]:
pbp_df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NaN,NaN,NaN,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.443521,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,1.468819,NaN,NaN,NaN,NaN,NaN,0.440373,-44.037291
3,89,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.492192,0.727261,6.988125,6.0,0.60693,0.227598,0.389904,61.009598
4,115,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.325931,NaN,NaN,NaN,NaN,NaN,0.443575,-44.357494


In [266]:
pbp_df.tail()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
110035,3976,2024_04_WAS_ARI,2024092908,ARI,WAS,REG,4,ARI,home,WAS,...,0,1,0.091620,NaN,NaN,NaN,NaN,NaN,0.833091,-83.309066
110036,4005,2024_04_WAS_ARI,2024092908,ARI,WAS,REG,4,ARI,home,WAS,...,0,1,0.184618,NaN,NaN,NaN,NaN,NaN,0.803180,-80.317980
110037,4027,2024_04_WAS_ARI,2024092908,ARI,WAS,REG,4,ARI,home,WAS,...,1,1,0.803026,NaN,NaN,NaN,NaN,NaN,0.865371,-86.537135
110038,4054,2024_04_WAS_ARI,2024092908,ARI,WAS,REG,4,ARI,home,WAS,...,0,1,-2.901049,NaN,NaN,NaN,NaN,NaN,0.920971,-92.097086
110039,4076,2024_04_WAS_ARI,2024092908,ARI,WAS,REG,4,ARI,home,WAS,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [267]:
qb_cols = ['player_id', 'player_name', 'player_display_name', 'position', 'recent_team', 'season', 'week',\
 'season_type', 'opponent_team', 'completions', 'attempts',\
 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',\
 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',\
 'passing_yards_after_catch', 'passing_epa',\
 'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',\
 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_2pt_conversions',\
 'receptions', 'receiving_yards', 'receiving_tds',\
 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_2pt_conversions', 'racr', 'fantasy_points', 'fantasy_points_ppr']

# Weekly data
This weekly data will enable us to derive the fantasy points of each QB in every game.

In [268]:
weekly_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12518 entries, 0 to 12517
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    12518 non-null  object 
 1   player_name                  12518 non-null  object 
 2   player_display_name          12518 non-null  object 
 3   position                     12518 non-null  object 
 4   position_group               12518 non-null  object 
 5   headshot_url                 12456 non-null  object 
 6   recent_team                  12518 non-null  object 
 7   season                       12518 non-null  int64  
 8   week                         12518 non-null  int64  
 9   season_type                  12518 non-null  object 
 10  opponent_team                12518 non-null  object 
 11  completions                  12518 non-null  int64  
 12  attempts                     12518 non-null  int64  
 13  passing_yards   

In [269]:
pbp_df = pbp_df.replace({'LA' : 'LAR', 'JAC' : 'JAX'})

In [270]:
weekly_df = weekly_df.replace({'LA' : 'LAR', 'JAC' : 'JAX'})

In [271]:
qb_df = weekly_df[weekly_df['position'] == 'QB']

In [272]:
qb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1540 entries, 0 to 12508
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    1540 non-null   object 
 1   player_name                  1540 non-null   object 
 2   player_display_name          1540 non-null   object 
 3   position                     1540 non-null   object 
 4   position_group               1540 non-null   object 
 5   headshot_url                 1535 non-null   object 
 6   recent_team                  1540 non-null   object 
 7   season                       1540 non-null   int64  
 8   week                         1540 non-null   int64  
 9   season_type                  1540 non-null   object 
 10  opponent_team                1540 non-null   object 
 11  completions                  1540 non-null   int64  
 12  attempts                     1540 non-null   int64  
 13  passing_yards         

In [273]:
qb_df = qb_df[qb_cols]

In [274]:
qb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1540 entries, 0 to 12508
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   player_id                  1540 non-null   object 
 1   player_name                1540 non-null   object 
 2   player_display_name        1540 non-null   object 
 3   position                   1540 non-null   object 
 4   recent_team                1540 non-null   object 
 5   season                     1540 non-null   int64  
 6   week                       1540 non-null   int64  
 7   season_type                1540 non-null   object 
 8   opponent_team              1540 non-null   object 
 9   completions                1540 non-null   int64  
 10  attempts                   1540 non-null   int64  
 11  passing_yards              1540 non-null   int64  
 12  passing_tds                1540 non-null   int64  
 13  interceptions              1540 non-null   int64  
 

In [275]:
fanduel_df = fanduel_df[fanduel_df['position'] == 'QB']
draftkings_df = draftkings_df[draftkings_df['position'] == 'QB']

# Setting up for concatenation
We need to add the current week's player list to the data that goes through last week so we can derive L8 variables for the current week. There are no DraftKings names that aren't in the FanDuel list, so we can just use the FanDuel list to concat.

In [276]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week
2,107566-62239,Josh Allen,QB,9300,BUF,HOU,0,Active,10-06-2024,5
5,107566-63115,Lamar Jackson,QB,8800,BAL,CIN,0,Active,10-06-2024,5
6,107566-102785,Jayden Daniels,QB,8700,WAS,CLE,1,Active,10-06-2024,5
10,107566-129471,CJ Stroud,QB,8400,HOU,BUF,1,Active,10-06-2024,5
13,107566-69017,Jordan Love,QB,8200,GB,LAR,0,Active,10-06-2024,5


In [277]:
def clean_name(name):
    # Remove periods between initials like C.J., D.J. (case-sensitive)
    name = re.sub(r'\b([A-Z])\.\s*([A-Z])\.\b', r'\1\2', name)
    
    # Remove common suffixes like Jr., Sr., III, II, IV (case-sensitive)
    cleaned_name = re.sub(r'(\,|\.|Sr|Jr|III|II|IV)', '', name).strip()
    
    return cleaned_name

In [278]:
qb_df['player_display_name'] = qb_df['player_display_name'].apply(clean_name)

In [279]:
from rapidfuzz import process, fuzz

In [280]:
SIMILARITY_THRESHOLD = 80

In [281]:
def fuzzy_match(name, dk_names):
    match, score, _ = process.extractOne(name, dk_names, scorer=fuzz.token_sort_ratio)
    return match if score >= SIMILARITY_THRESHOLD else None

In [282]:
qb_names = qb_df['player_display_name']

In [283]:
fanduel_df['matched_name'] = fanduel_df['name'].apply(lambda x: fuzzy_match(x, qb_names))

In [284]:
unmatched_in_fd = fanduel_df[fanduel_df['matched_name'].isna()]

In [285]:
#If all unmatched players have a salary of 6000, the QB minimum, then we don't need to worry about it
unmatched_in_fd[unmatched_in_fd['salary'] > 6000]

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name


In [286]:
fanduel_df[(fanduel_df['name'] != fanduel_df['matched_name']) & (fanduel_df['matched_name'].notna())]

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name


In [287]:
draftkings_df['matched_name'] = draftkings_df['name'].apply(lambda x: fuzzy_match(x, qb_names))

In [288]:
unmatched_in_dk = draftkings_df[draftkings_df['matched_name'].isna()]

In [289]:
unmatched_in_dk[unmatched_in_dk['salary'] > 4000]

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name


In [290]:
draftkings_df[(draftkings_df['name'] != draftkings_df['matched_name']) & (draftkings_df['matched_name'].notna())]

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name


In [291]:
# fanduel_df.replace({'Gardner Minshew II': 'Gardner Minshew', 'PJ Walker': 'P.J. Walker'}, inplace = True)
# draftkings_df.replace({'Gardner Minshew II': 'Gardner Minshew', 'PJ Walker': 'P.J. Walker'}, inplace = True)

In [292]:
new_df = fanduel_df[['name', 'position', 'team', 'opponent', 'week']]
new_df.rename(columns = {'name': 'player_display_name', 'team' : 'recent_team', 'opponent': 'opponent_team'}, inplace = True)
new_df['season'] = season

C:\Users\Owner\AppData\Local\Temp\ipykernel_6124\3609715700.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns = {'name': 'player_display_name', 'team' : 'recent_team', 'opponent': 'opponent_team'}, inplace = True)
C:\Users\Owner\AppData\Local\Temp\ipykernel_6124\3609715700.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['season'] = season


In [293]:
qb_df.drop(columns = ['player_id', 'player_name', 'season_type'], inplace = True)

In [294]:
qb_df = pd.concat([qb_df, new_df], axis = 0)

In [295]:
qb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1628 entries, 0 to 148
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   player_display_name        1628 non-null   object 
 1   position                   1628 non-null   object 
 2   recent_team                1628 non-null   object 
 3   season                     1628 non-null   int64  
 4   week                       1628 non-null   int64  
 5   opponent_team              1628 non-null   object 
 6   completions                1540 non-null   float64
 7   attempts                   1540 non-null   float64
 8   passing_yards              1540 non-null   float64
 9   passing_tds                1540 non-null   float64
 10  interceptions              1540 non-null   float64
 11  sacks                      1540 non-null   float64
 12  sack_yards                 1540 non-null   float64
 13  sack_fumbles               1540 non-null   float64
 14

In [296]:
qb_cols_to_group = ['season', 'week', 'player_display_name', 'recent_team', 'opponent_team']

In [297]:
qb_scoring_cols = ['passing_yards', 'passing_tds', 'interceptions', 'passing_2pt_conversions', 'rushing_yards',\
                   'rushing_tds', 'rushing_fumbles_lost', 'rushing_2pt_conversions', 'receptions', 'receiving_yards',\
                   'receiving_tds', 'receiving_fumbles_lost', 'receiving_2pt_conversions', 'sack_fumbles_lost' ]

In [298]:
qb_df = qb_df.set_index(qb_cols_to_group, drop = True)

# Fantasy points
This is where we calculate FanDuel and DraftKings points.

In [299]:
qb_df['FD_Pts'] = (qb_df['passing_yards'] * 0.04) + (qb_df['rushing_tds'] * 6) + (qb_df['rushing_yards'] * 0.1)\
+ (qb_df['passing_tds'] * 4) + (qb_df['receiving_yards'] * 0.1) + (qb_df['receiving_tds'] * 6) + (qb_df['receptions'] * 0.5)\
+ (qb_df['rushing_2pt_conversions'] * 2) + (qb_df['passing_2pt_conversions'] * 2) + (qb_df['receiving_2pt_conversions'] * 2)\
- (qb_df['interceptions']) - (qb_df['sack_fumbles_lost'] * 2) - (qb_df['receiving_fumbles_lost'] * 2) - (qb_df['rushing_fumbles_lost'] * 2)

In [300]:
qb_df['DK_Pts'] = (qb_df['passing_yards'] * 0.04) + (qb_df['rushing_tds'] * 6) + (qb_df['rushing_yards'] * 0.1)\
+ (qb_df['passing_tds'] * 4) + (qb_df['receiving_yards'] * 0.1) + (qb_df['receiving_tds'] * 6) + (qb_df['receptions'] * 1)\
+ (qb_df['rushing_2pt_conversions'] * 2) + (qb_df['passing_2pt_conversions'] * 2) + (qb_df['receiving_2pt_conversions'] * 2)\
- (qb_df['interceptions']) - (qb_df['sack_fumbles_lost']) - (qb_df['receiving_fumbles_lost']) - (qb_df['rushing_fumbles_lost'])


Adding DraftKings bonus points

In [301]:
qb_df['DK_Pts'] = np.where(qb_df['passing_yards'] >= 300, qb_df['DK_Pts'] + 3, qb_df['DK_Pts'])
qb_df['DK_Pts'] = np.where(qb_df['receiving_yards'] >= 100, qb_df['DK_Pts'] + 3, qb_df['DK_Pts'])
qb_df['DK_Pts'] = np.where(qb_df['rushing_yards'] >= 100, qb_df['DK_Pts'] + 3, qb_df['DK_Pts'])

In [302]:
#pacr = passing air conversion ratio (passing yards/air yards)

In [303]:
qb_df.reset_index(inplace = True)

In [304]:
qb_df.tail()

,season,week,player_display_name,recent_team,opponent_team,position,completions,attempts,passing_yards,passing_tds,...,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_2pt_conversions,racr,fantasy_points,fantasy_points_ppr,FD_Pts,DK_Pts
1623,2024,5,EJ Perry,JAX,IND,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1624,2024,5,Anthony Brown,BUF,HOU,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1625,2024,5,Trace McSorley,WAS,CLE,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1626,2024,5,Jason Bean,IND,JAX,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1627,2024,5,Ben DiNucci,BUF,HOU,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [305]:
#fanduel_df.head()

In [306]:
#draftkings_df.head()

In [307]:
#We need to group QBs by team because for now we're trying to get DvP means for teams against QBs
grouped_pts = qb_df.groupby(['season', 'week', 'recent_team', 'opponent_team'])[['DK_Pts', 'FD_Pts']].sum().round(3)

In [308]:
#grouped_pts[(grouped_pts['opponent_team'] == 'CLE') & (grouped_pts['season'] == 2001) & (grouped_pts['week'] == 11)]

In [309]:
grouped_pts.reset_index(inplace = True)

In [310]:
grouped_pts

,season,week,recent_team,opponent_team,DK_Pts,FD_Pts
0,2022,1,ARI,KC,23.10,23.10
1,2022,1,ATL,NO,20.80,19.80
2,2022,1,BAL,NYJ,21.22,21.22
3,2022,1,BUF,LAR,33.48,33.48
4,2022,1,CAR,CLE,19.00,19.00
...,...,...,...,...,...,...
1283,2024,5,NYG,SEA,0.00,0.00
1284,2024,5,PIT,DAL,0.00,0.00
1285,2024,5,SEA,NYG,0.00,0.00
1286,2024,5,SF,ARI,0.00,0.00


In [311]:
grouped_pts = grouped_pts.sort_values(by = ['opponent_team', 'season', 'week'])
grouped_pts['opp_game_num'] = grouped_pts.groupby(['opponent_team']).cumcount() + 1
grouped_pts.reset_index(inplace = True)

In [312]:
grouped_pts_raw = grouped_pts.copy()

In [313]:
grouped_pts.drop(columns = ['index'], inplace = True)

# DvP variables
We'll take an 8-game rolling mean for fantasy points allowed to QBs for each team, even if the games go back to last season. DvP means defense vs. position.

In [314]:
def calculate_equal_rolling_mean(group, cols, suffix):
    """
    This function calculates a rolling mean for the last eight games, going back to previous season if necessary.
    It also calculates when there are less than eight games to use.
    """
    for col in cols:
        group[f'{col}{suffix}'] = (
            group[col].shift().rolling(window=8, min_periods=1).mean()
        )
    return group

In [315]:
grouped_pts = grouped_pts.groupby('opponent_team', as_index = False).apply(calculate_equal_rolling_mean, cols=['FD_Pts', 'DK_Pts'], suffix = '_DvP')

In [316]:
grouped_pts.drop(columns = ['DK_Pts', 'FD_Pts', 'opp_game_num'], inplace = True)

In [317]:
qb_df = pd.merge(qb_df, grouped_pts, on = ['season', 'week', 'recent_team', 'opponent_team'], how = 'left')

In [318]:
qb_df.head()

,season,week,player_display_name,recent_team,opponent_team,position,completions,attempts,passing_yards,passing_tds,...,receiving_fumbles,receiving_fumbles_lost,receiving_2pt_conversions,racr,fantasy_points,fantasy_points_ppr,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP
0,2022,1,Tom Brady,TB,DAL,QB,18.0,27.0,212.0,1.0,...,0.0,0.0,0.0,NaN,10.38,10.38,11.38,11.38,NaN,NaN
1,2022,2,Tom Brady,TB,NO,QB,18.0,34.0,190.0,1.0,...,0.0,0.0,0.0,NaN,9.40,9.40,9.40,10.40,19.800000,20.800
2,2022,3,Tom Brady,TB,GB,QB,31.0,42.0,271.0,1.0,...,0.0,0.0,0.0,NaN,14.74,14.74,14.74,14.74,14.440000,14.440
3,2022,4,Tom Brady,TB,KC,QB,39.0,52.0,385.0,3.0,...,0.0,0.0,0.0,NaN,25.40,25.40,25.40,29.40,20.946667,22.280
4,2022,5,Tom Brady,TB,ATL,QB,35.0,52.0,351.0,1.0,...,0.0,0.0,0.0,NaN,19.74,19.74,19.74,22.74,23.300000,24.175


In [319]:
qb_points_last_week = qb_df[(qb_df['season'] == season) & (qb_df['week'] == week - 1)]\
[['season', 'week', 'player_display_name', 'recent_team', 'opponent_team', 'position', 'FD_Pts', 'DK_Pts']]

In [320]:
qb_points_last_week.to_csv('qb_target_' + str(season) + '_' + str(week - 1) + '.csv')

In [321]:
# conn = sqlite3.connect('nfl_dfs.db')

# # Specify the table name
# table_name = 'pbp_non_defense'

# # Query the table and load it into a pandas DataFrame
# pbp_df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)

# # Display the DataFrame
# print(pbp_df)

# # Close the connection
# conn.close()

In [322]:
pbp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110040 entries, 0 to 110039
Columns: 372 entries, play_id to pass_oe
dtypes: float64(182), int64(39), object(151)
memory usage: 312.3+ MB


In [323]:
qb_df.reset_index(inplace = True)

In [324]:
qb_df.rename(columns = {'recent_team': 'posteam'}, inplace = True)

# Introducing role variables
We brought in play-by-play data to derive our role variable. For games where more than one quarterback took a snap for a team, we're going to see which one took more snaps and give that QB a QB_role value of 1. All other QBs will have a value of 2. This technique will take on greater importance when we do RBs and WRs. In the case of QBs, if a backup quarterback becomes the starter, we'll change the role value to 1, and that should provide a prediction that's more reflective of the QBs greater role.<br>

In [325]:
# Assuming passer_df is already created
passer_df = pbp_df.groupby(['season', 'week', 'posteam', 'passer_player_id'])['play_id'].count().reset_index()

In [326]:
# Sort by season, week, posteam, and the number of plays in descending order
passer_df = passer_df.sort_values(by=['season', 'week', 'posteam', 'play_id'], ascending=[True, True, True, False])

In [327]:
# Create a rank column to identify the top two QBs
passer_df['QB_role'] = passer_df.groupby(['season', 'week', 'posteam'])['play_id'].rank(method='first', ascending=False)

In [328]:
# Assign 'QB1' or 'QB2' based on the rank
#passer_df['QB_role'] = passer_df['rank'].apply(lambda x: 'QB1' if x == 1 else ('QB2' if x == 2 else None))

In [329]:
# Drop the rank column if you don't need it anymore
#passer_df = passer_df.drop(columns=['rank'])

In [330]:
#passer_df.rename(columns = {'passer_player_id': 'player_id'}, inplace = True)

In [331]:
passer_df.drop(columns = ['play_id'], inplace = True)

In [332]:
passer_df.head()

,season,week,posteam,passer_player_id,QB_role
1,2022,1,ARI,00-0035228,1.0
0,2022,1,ARI,00-0035146,2.0
2,2022,1,ATL,00-0032268,1.0
3,2022,1,BAL,00-0034796,1.0
4,2022,1,BUF,00-0034857,1.0


In [333]:
passer_df.rename(columns = {'passer_player_id': 'player_id', 'posteam': 'recent_team'}, inplace = True)

We need to merge what we get from the play-by-play data back into the weekly data since the weekly data uses full names, and the pbp only uses first initial and last name.

In [334]:
merge_df = weekly_df[weekly_df['position'] == 'QB']

In [335]:
merge_df = merge_df[['player_id', 'season', 'week', 'recent_team', 'player_display_name']]

In [336]:
merge_df = pd.merge(merge_df, passer_df, on = ['season', 'week', 'recent_team', 'player_id'], how = 'left')

In [337]:
merge_df

,player_id,season,week,recent_team,player_display_name,QB_role
0,00-0019596,2022,1,TB,Tom Brady,1.0
1,00-0019596,2022,2,TB,Tom Brady,1.0
2,00-0019596,2022,3,TB,Tom Brady,1.0
3,00-0019596,2022,4,TB,Tom Brady,1.0
4,00-0019596,2022,5,TB,Tom Brady,1.0
...,...,...,...,...,...,...
1535,00-0039910,2024,4,WAS,Jayden Daniels,1.0
1536,00-0039918,2024,1,CHI,Caleb Williams,1.0
1537,00-0039918,2024,2,CHI,Caleb Williams,1.0
1538,00-0039918,2024,3,CHI,Caleb Williams,1.0


In [338]:
merge_df.rename(columns = {'recent_team' : 'posteam'}, inplace = True)
#merge_df.drop(columns = ['player_id'], inplace = True)

# Merging
Now we'll merge so that our QB_role variable is added to the dataframe. This is where we also change the name of the dataframe to **quarterback_df.**

In [339]:
quarterback_df = pd.merge(qb_df, merge_df, on = ['season', 'week', 'posteam', 'player_display_name'], how = 'left')

In [340]:
quarterback_df.tail()

,index,season,week,player_display_name,posteam,opponent_team,position,completions,attempts,passing_yards,...,receiving_2pt_conversions,racr,fantasy_points,fantasy_points_ppr,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP,player_id,QB_role
1623,1623,2024,5,EJ Perry,JAX,IND,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19.1125,20.1125,NaN,NaN
1624,1624,2024,5,Anthony Brown,BUF,HOU,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.5625,18.4375,NaN,NaN
1625,1625,2024,5,Trace McSorley,WAS,CLE,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15.4900,15.4900,NaN,NaN
1626,1626,2024,5,Jason Bean,IND,JAX,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.1700,18.9200,NaN,NaN
1627,1627,2024,5,Ben DiNucci,BUF,HOU,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.5625,18.4375,NaN,NaN


In [341]:
#quarterback_df = quarterback_df.dropna(subset=['QB_role'])

# Paring down some of the columns
We're drop columns that won't be needed for features.

In [342]:
cols_to_keep = ['season', 'week', 'player_id', 'player_display_name', 'posteam',
       'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds',
       'interceptions', 'sacks', 'sack_fumbles_lost', 'passing_air_yards', 
        'pacr',  'carries', 'rushing_yards', 'rushing_tds', 'FD_Pts', 'DK_Pts', 'FD_Pts_DvP', 'DK_Pts_DvP', 'QB_role']

In [343]:
quarterback_df = quarterback_df[cols_to_keep]

In [344]:
#quarterback_df[quarterback_df['season'] >= 2006]['passing_air_yards']

In [345]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1628 entries, 0 to 1627
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1628 non-null   int64  
 1   week                 1628 non-null   int64  
 2   player_id            1498 non-null   object 
 3   player_display_name  1628 non-null   object 
 4   posteam              1628 non-null   object 
 5   opponent_team        1628 non-null   object 
 6   completions          1540 non-null   float64
 7   attempts             1540 non-null   float64
 8   passing_yards        1540 non-null   float64
 9   passing_tds          1540 non-null   float64
 10  interceptions        1540 non-null   float64
 11  sacks                1540 non-null   float64
 12  sack_fumbles_lost    1540 non-null   float64
 13  passing_air_yards    1540 non-null   float64
 14  pacr                 1468 non-null   float64
 15  carries              1540 non-null   f

# Checking missing on previous weeks
Before we check missing values, we should temporarily break up the dataframe so that the current week, which we're trying to fill, is separate from the rest of the data. Then we can deal with missing values for previous weeks before bringing back the rows we want to fill.

In [346]:
qb_df_temp = quarterback_df[~((quarterback_df['season'] == season) & (quarterback_df['week'] == week))]

In [347]:
this_week = quarterback_df[(quarterback_df['season'] == season) & (quarterback_df['week'] == week)]

In [348]:
qb_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1540 entries, 0 to 1539
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1540 non-null   int64  
 1   week                 1540 non-null   int64  
 2   player_id            1498 non-null   object 
 3   player_display_name  1540 non-null   object 
 4   posteam              1540 non-null   object 
 5   opponent_team        1540 non-null   object 
 6   completions          1540 non-null   float64
 7   attempts             1540 non-null   float64
 8   passing_yards        1540 non-null   float64
 9   passing_tds          1540 non-null   float64
 10  interceptions        1540 non-null   float64
 11  sacks                1540 non-null   float64
 12  sack_fumbles_lost    1540 non-null   float64
 13  passing_air_yards    1540 non-null   float64
 14  pacr                 1468 non-null   float64
 15  carries              1540 non-null   float6

In [349]:
this_week.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, 1540 to 1627
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               88 non-null     int64  
 1   week                 88 non-null     int64  
 2   player_id            0 non-null      object 
 3   player_display_name  88 non-null     object 
 4   posteam              88 non-null     object 
 5   opponent_team        88 non-null     object 
 6   completions          0 non-null      float64
 7   attempts             0 non-null      float64
 8   passing_yards        0 non-null      float64
 9   passing_tds          0 non-null      float64
 10  interceptions        0 non-null      float64
 11  sacks                0 non-null      float64
 12  sack_fumbles_lost    0 non-null      float64
 13  passing_air_yards    0 non-null      float64
 14  pacr                 0 non-null      float64
 15  carries              0 non-null      float

In [350]:
# quarterback_df['passing_air_yards'] = np.where((quarterback_df['season'] <= 2005) & (quarterback_df['QB_role'] == 1),\
#                                                quarterback_df['passing_air_yards'].mean(), quarterback_df['passing_air_yards'])

# Filling pacr
pacr is passing yards/air yards, and every QB will have at least some air yards even if he doesn't have passing yards. So we were going to just fill missing pacr values with 0, but upon further review, we thought about filling with 1. Then we saw the mean is almost 1 (0.93) and the median is .86. So we'll just fill with the mean

In [351]:
qb_df_temp['pacr'] = qb_df_temp['pacr'].fillna(qb_df_temp['pacr'].mean())

C:\Users\Owner\AppData\Local\Temp\ipykernel_6124\52539329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qb_df_temp['pacr'] = qb_df_temp['pacr'].fillna(qb_df_temp['pacr'].mean())


Let's see if dropping all 2022 Week 1 rows will take care of some of the missing values. We obviously don't need that for L8 variables.

In [352]:
# quarterback_df['FD_Pts_DvP'] = quarterback_df['FD_Pts_DvP'].fillna(quarterback_df['FD_Pts_DvP'].mean())
# quarterback_df['DK_Pts_DvP'] = quarterback_df['DK_Pts_DvP'].fillna(quarterback_df['DK_Pts_DvP'].mean())
qb_df_temp = qb_df_temp[~((qb_df_temp['season'] == 2022) & (qb_df_temp['week'] == 1))]

In [353]:
qb_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1503 entries, 1 to 1539
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1503 non-null   int64  
 1   week                 1503 non-null   int64  
 2   player_id            1461 non-null   object 
 3   player_display_name  1503 non-null   object 
 4   posteam              1503 non-null   object 
 5   opponent_team        1503 non-null   object 
 6   completions          1503 non-null   float64
 7   attempts             1503 non-null   float64
 8   passing_yards        1503 non-null   float64
 9   passing_tds          1503 non-null   float64
 10  interceptions        1503 non-null   float64
 11  sacks                1503 non-null   float64
 12  sack_fumbles_lost    1503 non-null   float64
 13  passing_air_yards    1503 non-null   float64
 14  pacr                 1503 non-null   float64
 15  carries              1503 non-null   float6

In [354]:
this_week.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, 1540 to 1627
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               88 non-null     int64  
 1   week                 88 non-null     int64  
 2   player_id            0 non-null      object 
 3   player_display_name  88 non-null     object 
 4   posteam              88 non-null     object 
 5   opponent_team        88 non-null     object 
 6   completions          0 non-null      float64
 7   attempts             0 non-null      float64
 8   passing_yards        0 non-null      float64
 9   passing_tds          0 non-null      float64
 10  interceptions        0 non-null      float64
 11  sacks                0 non-null      float64
 12  sack_fumbles_lost    0 non-null      float64
 13  passing_air_yards    0 non-null      float64
 14  pacr                 0 non-null      float64
 15  carries              0 non-null      float

In [355]:
quarterback_df = pd.concat([qb_df_temp, this_week], axis = 0)

In [356]:
#With the dataframes back together, we can universally fill in missing QB_role values with 2
quarterback_df['QB_role'] = quarterback_df['QB_role'].fillna(2)

In [357]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1591 entries, 1 to 1627
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1591 non-null   int64  
 1   week                 1591 non-null   int64  
 2   player_id            1461 non-null   object 
 3   player_display_name  1591 non-null   object 
 4   posteam              1591 non-null   object 
 5   opponent_team        1591 non-null   object 
 6   completions          1503 non-null   float64
 7   attempts             1503 non-null   float64
 8   passing_yards        1503 non-null   float64
 9   passing_tds          1503 non-null   float64
 10  interceptions        1503 non-null   float64
 11  sacks                1503 non-null   float64
 12  sack_fumbles_lost    1503 non-null   float64
 13  passing_air_yards    1503 non-null   float64
 14  pacr                 1503 non-null   float64
 15  carries              1503 non-null   float6

In [358]:
quarterback_df.drop(columns = ['player_id'], inplace = True)

In [359]:
missing_df = quarterback_df[quarterback_df.isna().any(axis = 1)]

In [360]:
missing_df['season'].value_counts()

season
2024    88
Name: count, dtype: int64

In [361]:
missing_df['week'].value_counts()

week
5    88
Name: count, dtype: int64

In [362]:
cols_L8 = ['completions', 'attempts', 'passing_yards', 'passing_tds','interceptions',\
           'sacks', 'passing_air_yards', 'pacr', 'carries', 'rushing_yards', 'rushing_tds']

In [363]:
qb_L8_features = quarterback_df.groupby(['player_display_name', 'season', 'week'])[cols_L8].sum()

In [364]:
qb_L8_features

completions  attempts  passing_yards  \
player_display_name season week                                         
AJ McCarron         2023   14            1.0       1.0           -1.0   
                           18            3.0       4.0           20.0   
Aaron Rodgers       2022   2            19.0      25.0          234.0   
                           3            27.0      35.0          255.0   
                           4            21.0      35.0          251.0   
...                                      ...       ...            ...   
Zach Wilson         2023   10           23.0      39.0          263.0   
                           11            7.0      15.0           81.0   
                           14           27.0      36.0          301.0   
                           15            4.0      11.0           26.0   
                    2024   5             0.0       0.0            0.0   

                                 passing_tds  interceptions  sacks  \
player_display_name season week                                      
AJ McCarron         2023   14            0.0            0.0    0.0   
                           18            0.0            0.0    1.0   
Aaron Rodgers       2022   2             2.0            0.0    3.0   
                           3             2.0            1.0    1.0   
                           4             2.0            1.0    1.0   
...                                      ...            ...    ...   
Zach Wilson         2023   10            0.0            1.0    2.0   
                           11            1.0            1.0    5.0   
                           14            2.0            0.0    4.0   
                           15            0.0            0.0    4.0   
                    2024   5             0.0            0.0    0.0   

                                 passing_air_yards      pacr  carries  \
player_display_name season week                                         
AJ McCarron         2023   14                 -1.0  0.000000      0.0   
                           18                 -7.0  0.000000      0.0   
Aaron Rodgers       2022   2                 139.0  1.683453      5.0   
                           3                 177.0  1.440678      1.0   
                           4                 357.0  0.703081      1.0   
...                                            ...       ...      ...   
Zach Wilson         2023   10                358.0  0.734637      4.0   
                           11                 76.0  1.065789      4.0   
                           14                238.0  1.264706      3.0   
                           15                 29.0  0.896552      0.0   
                    2024   5                   0.0  0.000000      0.0   

                                 rushing_yards  rushing_tds  
player_display_name season week                              
AJ McCarron         2023   14              0.0          0.0  
                           18              0.0          0.0  
Aaron Rodgers       2022   2              10.0          0.0  
                           3              -1.0          0.0  
                           4               1.0          0.0  
...                                        ...          ...  
Zach Wilson         2023   10             54.0          0.0  
                           11             15.0          0.0  
                           14             12.0          0.0  
                           15              0.0          0.0  
                    2024   5               0.0          0.0  

[1591 rows x 11 columns]

# L8 variables
L8 variables are rolling means of features over the last eight games that each QB has played. Just like we did for the DvP variables, we'll calculate features over the previous eight games for individual QBs.

In [365]:
qb_L8_features = qb_L8_features.sort_values(by = ['player_display_name', 'season', 'week'])
qb_L8_features['game_num'] = qb_L8_features.groupby(['player_display_name', 'season']).cumcount() + 1
#quarterback_df.reset_index(drop = True, inplace = True)

In [366]:
qb_L8_features

completions  attempts  passing_yards  \
player_display_name season week                                         
AJ McCarron         2023   14            1.0       1.0           -1.0   
                           18            3.0       4.0           20.0   
Aaron Rodgers       2022   2            19.0      25.0          234.0   
                           3            27.0      35.0          255.0   
                           4            21.0      35.0          251.0   
...                                      ...       ...            ...   
Zach Wilson         2023   10           23.0      39.0          263.0   
                           11            7.0      15.0           81.0   
                           14           27.0      36.0          301.0   
                           15            4.0      11.0           26.0   
                    2024   5             0.0       0.0            0.0   

                                 passing_tds  interceptions  sacks  \
player_display_name season week                                      
AJ McCarron         2023   14            0.0            0.0    0.0   
                           18            0.0            0.0    1.0   
Aaron Rodgers       2022   2             2.0            0.0    3.0   
                           3             2.0            1.0    1.0   
                           4             2.0            1.0    1.0   
...                                      ...            ...    ...   
Zach Wilson         2023   10            0.0            1.0    2.0   
                           11            1.0            1.0    5.0   
                           14            2.0            0.0    4.0   
                           15            0.0            0.0    4.0   
                    2024   5             0.0            0.0    0.0   

                                 passing_air_yards      pacr  carries  \
player_display_name season week                                         
AJ McCarron         2023   14                 -1.0  0.000000      0.0   
                           18                 -7.0  0.000000      0.0   
Aaron Rodgers       2022   2                 139.0  1.683453      5.0   
                           3                 177.0  1.440678      1.0   
                           4                 357.0  0.703081      1.0   
...                                            ...       ...      ...   
Zach Wilson         2023   10                358.0  0.734637      4.0   
                           11                 76.0  1.065789      4.0   
                           14                238.0  1.264706      3.0   
                           15                 29.0  0.896552      0.0   
                    2024   5                   0.0  0.000000      0.0   

                                 rushing_yards  rushing_tds  game_num  
player_display_name season week                                        
AJ McCarron         2023   14              0.0          0.0         1  
                           18              0.0          0.0         2  
Aaron Rodgers       2022   2              10.0          0.0         1  
                           3              -1.0          0.0         2  
                           4               1.0          0.0         3  
...                                        ...          ...       ...  
Zach Wilson         2023   10             54.0          0.0         9  
                           11             15.0          0.0        10  
                           14             12.0          0.0        11  
                           15              0.0          0.0        12  
                    2024   5               0.0          0.0         1  

[1591 rows x 12 columns]

In [367]:
qb_L8_features = qb_L8_features.groupby(['player_display_name'], as_index = False).apply(calculate_equal_rolling_mean, cols=cols_L8, suffix = '_L8')

In [368]:
qb_L8_features.reset_index(inplace = True)

In [369]:
qb_L8_features

,level_0,player_display_name,season,week,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,attempts_L8,passing_yards_L8,passing_tds_L8,interceptions_L8,sacks_L8,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8
0,0,AJ McCarron,2023,14,1.0,1.0,-1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,AJ McCarron,2023,18,3.0,4.0,20.0,0.0,0.0,1.0,...,1.000,-1.000,0.000,0.000,0.000,-1.000,0.000000,0.000,0.000,0.0
2,1,Aaron Rodgers,2022,2,19.0,25.0,234.0,2.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Aaron Rodgers,2022,3,27.0,35.0,255.0,2.0,1.0,1.0,...,25.000,234.000,2.000,0.000,3.000,139.000,1.683453,5.000,10.000,0.0
4,1,Aaron Rodgers,2022,4,21.0,35.0,251.0,2.0,1.0,1.0,...,30.000,244.500,2.000,0.500,2.000,158.000,1.562066,3.000,4.500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,129,Zach Wilson,2023,10,23.0,39.0,263.0,0.0,1.0,2.0,...,33.375,200.000,0.625,0.625,3.875,246.375,0.875553,3.125,16.250,0.0
1587,129,Zach Wilson,2023,11,7.0,15.0,81.0,1.0,1.0,5.0,...,35.625,215.375,0.500,0.625,3.875,279.125,0.785091,3.125,22.250,0.0
1588,129,Zach Wilson,2023,14,27.0,36.0,301.0,2.0,0.0,4.0,...,34.125,204.250,0.500,0.375,4.125,254.500,0.840476,3.000,19.625,0.0
1589,129,Zach Wilson,2023,15,4.0,11.0,26.0,0.0,0.0,4.0,...,34.125,222.250,0.750,0.375,4.250,253.875,0.917803,3.250,21.000,0.0


In [370]:
qb_L8_features.drop(columns = ['level_0', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',\
                              'sacks', 'passing_air_yards', 'pacr', 'carries', 'rushing_yards', 'rushing_tds'], inplace = True)

In [371]:
quarterback_df = pd.merge(quarterback_df, qb_L8_features, on = ['player_display_name', 'season', 'week'], how = 'left')

In [372]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1591 non-null   int64  
 1   week                  1591 non-null   int64  
 2   player_display_name   1591 non-null   object 
 3   posteam               1591 non-null   object 
 4   opponent_team         1591 non-null   object 
 5   completions           1503 non-null   float64
 6   attempts              1503 non-null   float64
 7   passing_yards         1503 non-null   float64
 8   passing_tds           1503 non-null   float64
 9   interceptions         1503 non-null   float64
 10  sacks                 1503 non-null   float64
 11  sack_fumbles_lost     1503 non-null   float64
 12  passing_air_yards     1503 non-null   float64
 13  pacr                  1503 non-null   float64
 14  carries               1503 non-null   float64
 15  rushing_yards        

In [373]:
quarterback_df[(quarterback_df.isna().any(axis = 1))]

,season,week,player_display_name,posteam,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,...,attempts_L8,passing_yards_L8,passing_tds_L8,interceptions_L8,sacks_L8,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8
0,2022,2,Tom Brady,TB,NO,18.0,34.0,190.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,2022,2,Aaron Rodgers,GB,CHI,19.0,25.0,234.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,2022,2,Matt Ryan,IND,JAX,16.0,30.0,195.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2022,2,Joe Flacco,NYJ,CLE,26.0,44.0,307.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,2022,7,Chad Henne,KC,SF,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,2024,5,EJ Perry,JAX,IND,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1587,2024,5,Anthony Brown,BUF,HOU,NaN,NaN,NaN,NaN,NaN,...,24.50,151.0,0.0,1.00,2.5,200.50,0.563588,1.5,-2.50,0.0
1588,2024,5,Trace McSorley,WAS,CLE,NaN,NaN,NaN,NaN,NaN,...,19.75,100.0,0.0,1.25,0.5,156.75,0.631779,3.0,10.25,0.0
1589,2024,5,Jason Bean,IND,JAX,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [374]:
quarterback_df.drop(columns = ['completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',\
                              'sacks', 'passing_air_yards', 'pacr', 'carries', 'rushing_yards', 'rushing_tds',\
                              'sack_fumbles_lost', 'game_num'], inplace = True)

In [375]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1591 non-null   int64  
 1   week                  1591 non-null   int64  
 2   player_display_name   1591 non-null   object 
 3   posteam               1591 non-null   object 
 4   opponent_team         1591 non-null   object 
 5   FD_Pts                1503 non-null   float64
 6   DK_Pts                1503 non-null   float64
 7   FD_Pts_DvP            1591 non-null   float64
 8   DK_Pts_DvP            1591 non-null   float64
 9   QB_role               1591 non-null   float64
 10  completions_L8        1461 non-null   float64
 11  attempts_L8           1461 non-null   float64
 12  passing_yards_L8      1461 non-null   float64
 13  passing_tds_L8        1461 non-null   float64
 14  interceptions_L8      1461 non-null   float64
 15  sacks_L8             

In [376]:
cols_to_check = ['completions_L8', 'attempts_L8', 'passing_yards_L8', 'passing_tds_L8',\
                 'interceptions_L8', 'sacks_L8', 'passing_air_yards_L8', 'pacr_L8',\
                 'carries_L8', 'rushing_yards_L8', 'rushing_tds_L8']

In [377]:
missing_L8 = quarterback_df[quarterback_df[cols_to_check].isna().any(axis = 1)]

In [378]:
#pd.set_option('display.max_rows', None)

In [379]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1591 non-null   int64  
 1   week                  1591 non-null   int64  
 2   player_display_name   1591 non-null   object 
 3   posteam               1591 non-null   object 
 4   opponent_team         1591 non-null   object 
 5   FD_Pts                1503 non-null   float64
 6   DK_Pts                1503 non-null   float64
 7   FD_Pts_DvP            1591 non-null   float64
 8   DK_Pts_DvP            1591 non-null   float64
 9   QB_role               1591 non-null   float64
 10  completions_L8        1461 non-null   float64
 11  attempts_L8           1461 non-null   float64
 12  passing_yards_L8      1461 non-null   float64
 13  passing_tds_L8        1461 non-null   float64
 14  interceptions_L8      1461 non-null   float64
 15  sacks_L8             

# Must fill missing values
We need to run the new data through the model, so we can't have missing data. We'll fill with the mean.

In [380]:
columns_to_fill = [
    'completions_L8', 'attempts_L8', 'passing_yards_L8', 'passing_tds_L8',
    'interceptions_L8', 'sacks_L8', 'passing_air_yards_L8', 'pacr_L8',
    'carries_L8', 'rushing_yards_L8', 'rushing_tds_L8'
]

In [381]:
quarterback_df[columns_to_fill] = quarterback_df[columns_to_fill].apply(lambda col: col.fillna(col.mean()))

In [382]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1591 non-null   int64  
 1   week                  1591 non-null   int64  
 2   player_display_name   1591 non-null   object 
 3   posteam               1591 non-null   object 
 4   opponent_team         1591 non-null   object 
 5   FD_Pts                1503 non-null   float64
 6   DK_Pts                1503 non-null   float64
 7   FD_Pts_DvP            1591 non-null   float64
 8   DK_Pts_DvP            1591 non-null   float64
 9   QB_role               1591 non-null   float64
 10  completions_L8        1591 non-null   float64
 11  attempts_L8           1591 non-null   float64
 12  passing_yards_L8      1591 non-null   float64
 13  passing_tds_L8        1591 non-null   float64
 14  interceptions_L8      1591 non-null   float64
 15  sacks_L8             

# Percentage variables
We'll take our L8 means and derive certain percentages.

In [383]:
quarterback_df['comp_pct'] = quarterback_df['completions_L8']/quarterback_df['attempts_L8']
quarterback_df['yds_per_attempt'] = quarterback_df['passing_yards_L8']/quarterback_df['attempts_L8']
quarterback_df['td_pct'] = quarterback_df['passing_tds_L8']/quarterback_df['attempts_L8']
quarterback_df['sack_pct'] = quarterback_df['sacks_L8']/quarterback_df['attempts_L8']
quarterback_df['int_pct'] = quarterback_df['interceptions_L8']/quarterback_df['attempts_L8']

In [384]:
quarterback_df.tail(10)

,season,week,player_display_name,posteam,opponent_team,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP,QB_role,...,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8,comp_pct,yds_per_attempt,td_pct,sack_pct,int_pct
1581,2024,5,Bryce Young,CAR,CHI,NaN,NaN,14.0900,15.2150,2.0,...,273.875000,0.671374,2.375000,13.750000,0.125000,0.557940,5.343348,0.008584,0.120172,0.021459
1582,2024,5,Josh Johnson,BAL,CIN,NaN,NaN,16.8500,17.3500,2.0,...,23.000000,2.034826,1.333333,1.666667,0.000000,0.562500,5.312500,0.000000,0.125000,0.000000
1583,2024,5,PJ Walker,SEA,NYG,NaN,NaN,16.9750,18.1625,2.0,...,184.625000,0.636713,2.250000,8.750000,0.000000,0.497110,6.404624,0.011561,0.075145,0.046243
1584,2024,5,Kyle Allen,PIT,DAL,NaN,NaN,16.7700,16.8950,2.0,...,70.000000,0.928603,2.375000,0.000000,0.000000,0.589744,5.333333,0.025641,0.064103,0.051282
1585,2024,5,Carter Bradley,LV,DEN,NaN,NaN,16.0825,16.2075,2.0,...,215.348113,0.961580,3.662746,16.100802,0.183676,0.644195,7.029313,0.041089,0.072998,0.022520
1586,2024,5,EJ Perry,JAX,IND,NaN,NaN,19.1125,20.1125,2.0,...,215.348113,0.961580,3.662746,16.100802,0.183676,0.644195,7.029313,0.041089,0.072998,0.022520
1587,2024,5,Anthony Brown,BUF,HOU,NaN,NaN,17.5625,18.4375,2.0,...,200.500000,0.563588,1.500000,-2.500000,0.000000,0.448980,6.163265,0.000000,0.102041,0.040816
1588,2024,5,Trace McSorley,WAS,CLE,NaN,NaN,15.4900,15.4900,2.0,...,156.750000,0.631779,3.000000,10.250000,0.000000,0.544304,5.063291,0.000000,0.025316,0.063291
1589,2024,5,Jason Bean,IND,JAX,NaN,NaN,18.1700,18.9200,2.0,...,215.348113,0.961580,3.662746,16.100802,0.183676,0.644195,7.029313,0.041089,0.072998,0.022520
1590,2024,5,Ben DiNucci,BUF,HOU,NaN,NaN,17.5625,18.4375,2.0,...,215.348113,0.961580,3.662746,16.100802,0.183676,0.644195,7.029313,0.041089,0.072998,0.022520


In [385]:
quarterback_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [386]:
mean_dict = quarterback_df.mean(numeric_only=True).to_dict()

In [387]:
mean_dict

{'season': 2022.7517284726587,
 'week': 9.241357636706473,
 'FD_Pts': 13.997764471057884,
 'DK_Pts': 14.670751829673986,
 'FD_Pts_DvP': 17.127334799916195,
 'DK_Pts_DvP': 17.95340630331328,
 'QB_role': 1.2451288497800126,
 'completions_L8': 17.857375085557834,
 'attempts_L8': 27.720470323653075,
 'passing_yards_L8': 194.85586030442292,
 'passing_tds_L8': 1.1390135588800887,
 'interceptions_L8': 0.6242527948893452,
 'sacks_L8': 2.0235316645480914,
 'passing_air_yards_L8': 215.34811283856456,
 'pacr_L8': 0.9615803028857308,
 'carries_L8': 3.662745673217952,
 'rushing_yards_L8': 16.10080179915909,
 'rushing_tds_L8': 0.18367556468172483,
 'comp_pct': 0.6384022778233298,
 'yds_per_attempt': 7.0019283651121205,
 'td_pct': 0.041031020009790156,
 'sack_pct': 0.07853229626188044,
 'int_pct': 0.023976066249085635}

In [388]:
columns_to_fill = ['comp_pct', 'yds_per_attempt', 'td_pct', 'sack_pct', 'int_pct']

In [389]:
quarterback_df[columns_to_fill] = quarterback_df[columns_to_fill].apply(lambda col: col.fillna(col.mean()))

In [390]:
# quarterback_df.drop(columns = ['completions', 'attempts', 'passing_yards', 'passing_tds', 'sacks', 'interceptions',\
#                               'sack_fumbles_lost', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'game_num'], inplace = True)

In [391]:
quarterback_df.tail(10)

,season,week,player_display_name,posteam,opponent_team,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP,QB_role,...,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8,comp_pct,yds_per_attempt,td_pct,sack_pct,int_pct
1581,2024,5,Bryce Young,CAR,CHI,NaN,NaN,14.0900,15.2150,2.0,...,273.875000,0.671374,2.375000,13.750000,0.125000,0.557940,5.343348,0.008584,0.120172,0.021459
1582,2024,5,Josh Johnson,BAL,CIN,NaN,NaN,16.8500,17.3500,2.0,...,23.000000,2.034826,1.333333,1.666667,0.000000,0.562500,5.312500,0.000000,0.125000,0.000000
1583,2024,5,PJ Walker,SEA,NYG,NaN,NaN,16.9750,18.1625,2.0,...,184.625000,0.636713,2.250000,8.750000,0.000000,0.497110,6.404624,0.011561,0.075145,0.046243
1584,2024,5,Kyle Allen,PIT,DAL,NaN,NaN,16.7700,16.8950,2.0,...,70.000000,0.928603,2.375000,0.000000,0.000000,0.589744,5.333333,0.025641,0.064103,0.051282
1585,2024,5,Carter Bradley,LV,DEN,NaN,NaN,16.0825,16.2075,2.0,...,215.348113,0.961580,3.662746,16.100802,0.183676,0.644195,7.029313,0.041089,0.072998,0.022520
1586,2024,5,EJ Perry,JAX,IND,NaN,NaN,19.1125,20.1125,2.0,...,215.348113,0.961580,3.662746,16.100802,0.183676,0.644195,7.029313,0.041089,0.072998,0.022520
1587,2024,5,Anthony Brown,BUF,HOU,NaN,NaN,17.5625,18.4375,2.0,...,200.500000,0.563588,1.500000,-2.500000,0.000000,0.448980,6.163265,0.000000,0.102041,0.040816
1588,2024,5,Trace McSorley,WAS,CLE,NaN,NaN,15.4900,15.4900,2.0,...,156.750000,0.631779,3.000000,10.250000,0.000000,0.544304,5.063291,0.000000,0.025316,0.063291
1589,2024,5,Jason Bean,IND,JAX,NaN,NaN,18.1700,18.9200,2.0,...,215.348113,0.961580,3.662746,16.100802,0.183676,0.644195,7.029313,0.041089,0.072998,0.022520
1590,2024,5,Ben DiNucci,BUF,HOU,NaN,NaN,17.5625,18.4375,2.0,...,215.348113,0.961580,3.662746,16.100802,0.183676,0.644195,7.029313,0.041089,0.072998,0.022520


# Wind speed, division games, odds
We'll bring in the odds CSVs. This is also where we can break off the data for the current week. We no longer need previous data to derive features.

In [392]:
fd_spreads = pd.read_csv('fd_spreads_' + str(season) + '_' + str(week) + '.csv')
dk_spreads = pd.read_csv('dk_spreads_' + str(season) + '_' + str(week) + '.csv')

In [393]:
fd_spreads.drop(columns = ['Unnamed: 0'], inplace = True)
dk_spreads.drop(columns = ['Unnamed: 0'], inplace = True)

In [394]:
fd_spreads.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,MIN,NYJ,40.5,0,1,0,-2.5,19.00,21.50,0,2024,5
1,CIN,BAL,48.5,1,0,0,2.5,25.50,23.00,12,2024,5
2,HOU,BUF,47.5,0,0,0,-1.5,23.00,24.50,0,2024,5
3,CHI,CAR,40.5,1,1,0,-3.5,18.50,22.00,17,2024,5
4,WAS,CLE,43.5,1,1,0,-3.0,20.25,23.25,5,2024,5


In [395]:
dk_spreads.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,MIN,NYJ,40.5,0,1,0,-2.0,19.25,21.25,0,2024,5
1,CIN,BAL,48.5,1,0,0,2.5,25.50,23.00,12,2024,5
2,HOU,BUF,47.0,0,0,0,-1.0,23.00,24.00,0,2024,5
3,CHI,CAR,41.0,1,1,0,-4.0,18.50,22.50,17,2024,5
4,WAS,CLE,43.5,1,1,0,-3.0,20.25,23.25,5,2024,5


In [396]:
quarterback_df.rename(columns = {'posteam': 'team', 'opponent_team': 'opponent'}, inplace = True)

In [397]:
quarterback_df = quarterback_df[(quarterback_df['season'] == season) & (quarterback_df['week'] == week)]

In [398]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, 1503 to 1590
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                88 non-null     int64  
 1   week                  88 non-null     int64  
 2   player_display_name   88 non-null     object 
 3   team                  88 non-null     object 
 4   opponent              88 non-null     object 
 5   FD_Pts                0 non-null      float64
 6   DK_Pts                0 non-null      float64
 7   FD_Pts_DvP            88 non-null     float64
 8   DK_Pts_DvP            88 non-null     float64
 9   QB_role               88 non-null     float64
 10  completions_L8        88 non-null     float64
 11  attempts_L8           88 non-null     float64
 12  passing_yards_L8      88 non-null     float64
 13  passing_tds_L8        88 non-null     float64
 14  interceptions_L8      88 non-null     float64
 15  sacks_L8              88 

In [399]:
qb_df_fd = pd.merge(quarterback_df, fd_spreads, on = ['team', 'opponent', 'season', 'week'], how = 'left')
qb_df_dk = pd.merge(quarterback_df, dk_spreads, on = ['team', 'opponent', 'season', 'week'], how = 'left')

In [400]:
qb_df_fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                88 non-null     int64  
 1   week                  88 non-null     int64  
 2   player_display_name   88 non-null     object 
 3   team                  88 non-null     object 
 4   opponent              88 non-null     object 
 5   FD_Pts                0 non-null      float64
 6   DK_Pts                0 non-null      float64
 7   FD_Pts_DvP            88 non-null     float64
 8   DK_Pts_DvP            88 non-null     float64
 9   QB_role               88 non-null     float64
 10  completions_L8        88 non-null     float64
 11  attempts_L8           88 non-null     float64
 12  passing_yards_L8      88 non-null     float64
 13  passing_tds_L8        88 non-null     float64
 14  interceptions_L8      88 non-null     float64
 15  sacks_L8              88 

In [401]:
divisions = {
    'CLE': 'AFC North',
    'LAR': 'NFC West',
    'LV': 'AFC West',
    'KC': 'AFC West',
    'CAR': 'NFC South',
    'NYG': 'NFC East',
    'HOU': 'AFC South',
    'DEN': 'AFC West',
    'MIN': 'NFC North',
    'TEN': 'AFC South',
    'JAX': 'AFC South',
    'SEA': 'NFC West',
    'DET': 'NFC North',
    'NO': 'NFC South',
    'CIN': 'AFC North',
    'ATL': 'NFC South',
    'NYJ': 'AFC East',
    'PHI': 'NFC East',
    'DAL': 'NFC East',
    'WAS': 'NFC East',
    'PIT': 'AFC North',
    'ARI': 'NFC West',
    'CHI': 'NFC North',
    'MIA': 'AFC East',
    'BUF': 'AFC East',
    'BAL': 'AFC North',
    'TB': 'NFC South',
    'SF': 'NFC West',
    'LAC': 'AFC West',
    'IND': 'AFC South',
    'GB': 'NFC North',
    'NE': 'AFC East'
}

# You now have a dictionary `divisions` where each team is mapped to its division.


# Adding div_game binary column

In [402]:
# Map the team and opponent columns to their respective divisions
qb_df_fd['team_division'] = qb_df_fd['team'].map(divisions)
qb_df_fd['opponent_division'] = qb_df_fd['opponent'].map(divisions)

# Create the div_game column (1 if they are in the same division, 0 otherwise)
qb_df_fd['div_game'] = np.where(qb_df_fd['team_division'] == qb_df_fd['opponent_division'], 1, 0)

# Optionally, drop the temporary division columns if you don't need them
qb_df_fd.drop(['team_division', 'opponent_division'], axis=1, inplace=True)


In [403]:
# Map the team and opponent columns to their respective divisions
qb_df_dk['team_division'] = qb_df_dk['team'].map(divisions)
qb_df_dk['opponent_division'] = qb_df_dk['opponent'].map(divisions)

# Create the div_game column (1 if they are in the same division, 0 otherwise)
qb_df_dk['div_game'] = np.where(qb_df_dk['team_division'] == qb_df_dk['opponent_division'], 1, 0)

# Optionally, drop the temporary division columns if you don't need them
qb_df_dk.drop(['team_division', 'opponent_division'], axis=1, inplace=True)

In [404]:
qb_df_fd.rename(columns = {'player_display_name': 'name'}, inplace = True)
qb_df_dk.rename(columns = {'player_display_name': 'name'}, inplace = True)

In [405]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name
2,107566-62239,Josh Allen,QB,9300,BUF,HOU,0,Active,10-06-2024,5,Josh Allen
5,107566-63115,Lamar Jackson,QB,8800,BAL,CIN,0,Active,10-06-2024,5,Lamar Jackson
6,107566-102785,Jayden Daniels,QB,8700,WAS,CLE,1,Active,10-06-2024,5,Jayden Daniels
10,107566-129471,CJ Stroud,QB,8400,HOU,BUF,1,Active,10-06-2024,5,CJ Stroud
13,107566-69017,Jordan Love,QB,8200,GB,LAR,0,Active,10-06-2024,5,Jordan Love


In [406]:
draftkings_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name
3,36141829,Josh Allen,QB,7700,BUF,HOU,0,Active,10-06-2024,5,Josh Allen
8,36141830,Lamar Jackson,QB,7500,BAL,CIN,0,Active,10-06-2024,5,Lamar Jackson
13,36141831,CJ Stroud,QB,7200,HOU,BUF,1,Active,10-06-2024,5,CJ Stroud
20,36141832,Jayden Daniels,QB,6800,WAS,CLE,1,Active,10-06-2024,5,Jayden Daniels
24,36141833,Kyler Murray,QB,6700,ARI,SF,0,Active,10-06-2024,5,Kyler Murray


In [407]:
fanduel_df.rename(columns = {'player_display_name': 'name'}, inplace = True)
draftkings_df.rename(columns = {'player_display_name': 'name'}, inplace = True)

In [408]:
# starting_qbs = [('BUF', 'Josh Allen'), ('MIA', 'Skylar Thompson'), ('NYJ', 'Aaron Rodgers'), ('NE', 'Jacoby Brissett'),\
#                ('BAL', 'Lamar Jackson'), ('CIN', 'Joe Burrow'), ('CLE', 'Deshaun Watson'), ('PIT', 'Justin Fields'),\
#                ('IND', 'Anthony Richardson'), ('JAX', 'Trevor Lawrence'), ('HOU', 'C.J. Stroud'), ('TEN', 'Will Levis'),\
#                ('KC', 'Patrick Mahomes'), ('LV', 'Gardner Minshew'), ('LAC', 'Justin Herbert'), ('DEN', 'Bo Nix'),\
#                ('DAL', 'Dak Prescott'), ('PHI', 'Jalen Hurts'), ('NYG', 'Daniel Jones'), ('WAS', 'Jayden Daniels'),\
#                ('MIN', 'Sam Darnold'), ('DET', 'Jared Goff'), ('GB', 'Malik Willis'), ('CHI', 'Caleb Williams'),\
#                ('TB', 'Baker Mayfield'), ('NO', 'Derek Carr'), ('CAR', 'Bryce Young'), ('ATL', 'Kirk Cousins'),\
#                ('SF', 'Brock Purdy'), ('SEA', 'Geno Smith'), ('LAR', 'Matthew Stafford'), ('ARI', 'Kyler Murray')]

In [409]:
# starting_qbs = pd.DataFrame(data = starting_qbs, columns = ['team', 'name'])

In [410]:
# starting_qbs.to_csv('starting_QBs.csv', index = False)

In [411]:
qb_1 = pd.read_csv('starting_QBs.csv')

In [412]:
qb_df_fd.loc[
    qb_df_fd.set_index(['team', 'name']).index.isin(qb_1.set_index(['team', 'name']).index), 
    'QB_role'
] = 1

In [413]:
qb_df_dk.loc[
    qb_df_dk.set_index(['team', 'name']).index.isin(qb_1.set_index(['team', 'name']).index), 
    'QB_role'
] = 1

In [414]:
features = ['FD_Pts_DvP', 'DK_Pts_DvP', 'QB_role', 'completions_L8', 'attempts_L8',
       'passing_yards_L8', 'passing_tds_L8', 'interceptions_L8', 'sacks_L8',
       'passing_air_yards_L8', 'pacr_L8', 'carries_L8', 'rushing_yards_L8',
       'rushing_tds_L8', 'comp_pct', 'yds_per_attempt', 'td_pct', 'sack_pct',
       'int_pct', 'wind', 'div_game', 'spread_line', 'total_line', 'outdoors',
       'grass', 'home_team', 'pred_total', 'opp_total']

In [415]:
qb_df_fd.drop(columns = ['FD_Pts', 'DK_Pts'], inplace = True)
qb_df_dk.drop(columns = ['FD_Pts', 'DK_Pts'], inplace = True)

In [416]:
qb_df_fd.replace([np.inf, -np.inf], np.nan, inplace=True)
qb_df_dk.replace([np.inf, -np.inf], np.nan, inplace=True)

In [417]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name
2,107566-62239,Josh Allen,QB,9300,BUF,HOU,0,Active,10-06-2024,5,Josh Allen
5,107566-63115,Lamar Jackson,QB,8800,BAL,CIN,0,Active,10-06-2024,5,Lamar Jackson
6,107566-102785,Jayden Daniels,QB,8700,WAS,CLE,1,Active,10-06-2024,5,Jayden Daniels
10,107566-129471,CJ Stroud,QB,8400,HOU,BUF,1,Active,10-06-2024,5,CJ Stroud
13,107566-69017,Jordan Love,QB,8200,GB,LAR,0,Active,10-06-2024,5,Jordan Love


In [418]:
fanduel_df = fanduel_df[['name', 'position', 'salary', 'team', 'opponent', 'status', 'week']]
draftkings_df = draftkings_df[['name', 'position', 'salary', 'team', 'opponent', 'status', 'week']]

In [419]:
fanduel_df.head()

,name,position,salary,team,opponent,status,week
2,Josh Allen,QB,9300,BUF,HOU,Active,5
5,Lamar Jackson,QB,8800,BAL,CIN,Active,5
6,Jayden Daniels,QB,8700,WAS,CLE,Active,5
10,CJ Stroud,QB,8400,HOU,BUF,Active,5
13,Jordan Love,QB,8200,GB,LAR,Active,5


In [420]:
draftkings_df.head()

,name,position,salary,team,opponent,status,week
3,Josh Allen,QB,7700,BUF,HOU,Active,5
8,Lamar Jackson,QB,7500,BAL,CIN,Active,5
13,CJ Stroud,QB,7200,HOU,BUF,Active,5
20,Jayden Daniels,QB,6800,WAS,CLE,Active,5
24,Kyler Murray,QB,6700,ARI,SF,Active,5


# Merging player list with the model features
We need to bring back the salary and status columns. This also will filter out players who aren't on the list for one site or the other.

In [421]:
qb_df_fd = pd.merge(fanduel_df, qb_df_fd, on = ['name', 'team', 'opponent', 'week'], how = 'left')

In [422]:
qb_df_dk = pd.merge(draftkings_df, qb_df_dk, on = ['name', 'team', 'opponent', 'week'], how = 'left')

In [423]:
qb_df_fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  88 non-null     object 
 1   position              88 non-null     object 
 2   salary                88 non-null     int64  
 3   team                  88 non-null     object 
 4   opponent              88 non-null     object 
 5   status                88 non-null     object 
 6   week                  88 non-null     int64  
 7   season                88 non-null     int64  
 8   FD_Pts_DvP            88 non-null     float64
 9   DK_Pts_DvP            88 non-null     float64
 10  QB_role               88 non-null     float64
 11  completions_L8        88 non-null     float64
 12  attempts_L8           88 non-null     float64
 13  passing_yards_L8      88 non-null     float64
 14  passing_tds_L8        88 non-null     float64
 15  interceptions_L8      88 

In [424]:
qb_df_dk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  65 non-null     object 
 1   position              65 non-null     object 
 2   salary                65 non-null     int64  
 3   team                  65 non-null     object 
 4   opponent              65 non-null     object 
 5   status                65 non-null     object 
 6   week                  65 non-null     int64  
 7   season                65 non-null     int64  
 8   FD_Pts_DvP            65 non-null     float64
 9   DK_Pts_DvP            65 non-null     float64
 10  QB_role               65 non-null     float64
 11  completions_L8        65 non-null     float64
 12  attempts_L8           65 non-null     float64
 13  passing_yards_L8      65 non-null     float64
 14  passing_tds_L8        65 non-null     float64
 15  interceptions_L8      65 

In [425]:
qb_df_fd = qb_df_fd.set_index(['name', 'team', 'position', 'salary', 'opponent', 'status', 'season', 'week'], drop = True)
qb_df_dk = qb_df_dk.set_index(['name', 'team', 'position', 'salary', 'opponent', 'status', 'season', 'week'], drop = True)

In [426]:
qb_df_fd = qb_df_fd[features]
qb_df_dk = qb_df_dk[features]

In [427]:
qb_df_fd.to_csv('FD_qb_for_model_' + str(season) + '_' + str(week) + '.csv')
qb_df_dk.to_csv('DK_qb_for_model_' + str(season) + '_' + str(week) + '.csv')

In [428]:
###FANDUEL SCORING
#Rushing yards made = 0.1pts	
#Rushing touchdowns = 6pts	
#Passing yards = 0.04pts	
#Passing touchdowns = 4pts	
#Interceptions = -1pt	
#Receiving yards = 0.1pts	
#Receiving touchdowns = 6pts	
#Receptions = 0.5pts	
#Kickoff return touchdowns = 6pts	
#Punt return touchdowns = 6pts	
#Fumbles lost = -2pts	
#Own fumbles recovered touchdowns = 6pts	
#Two-point conversions scored = 2pts	
#Two-point conversion passes = 2pts	
#Field-goals from 0-39 yards = 3pts	
#Field-goals from 40-49 yards = 4pts	
#Field-goals from 50+ yards = 5pts	
#Extra-point conversions = 1pt

###DRAFTKINGS SCORING
#PAssing TD = 4 pts
#passing yards = .04 pts
#300 passing yards = 3 pts (bonus)
#Interception = -1 pts
#Rushing TD = 6 pts
#Rushing yds = 0.1 pts
#100 yd rushing game = 3 pts (bonus)
#Receiving TD = 6 pts
#Receiving yds = 0.1 pts
#100 receiving yards game = 3 pts (bonus)
#Receptions = 1 pt
#Punt/kickoff/FG return for TD = 6 pts
#Fumble lost = -1 pt
#2 pt conversion (pass, run or catch) = 2 pts
#Offensive fumble recovery TD = 6

In [429]:
# #import sqlite3

# # Establish a connection to the SQLite database
# conn = sqlite3.connect('nfl_dfs.db')

# # Define the table name
# table_name = 'qb_dataset'

# # Specify data types
# dtype = {
#     'season': 'INTEGER',
#     'week': 'INTEGER',
#     'player_id': 'TEXT',
#     'player_display_name': 'TEXT',
#     'team': 'TEXT',
#     'FD_Pts': 'REAL',
#     'DK_Pts': 'REAL',
#     'FD_Pts_DvP': 'REAL',
#     'DK_Pts_DvP': 'REAL',
#     'QB_role': 'REAL',
#     'completions_L8': 'REAL',
#     'attempts_L8': 'REAL',
#     'passing_yards_L8': 'REAL',
#     'passing_tds_L8': 'REAL',
#     'interceptions_L8': 'REAL',
#     'sacks_L8': 'REAL',
#     'passing_air_yards_L8': 'REAL',
#     'pacr_L8': 'REAL',
#     'carries_L8': 'REAL',
#     'rushing_yards_L8': 'REAL',
#     'rushing_tds_L8': 'REAL',
#     'comp_pct': 'REAL',
#     'yds_per_attempt': 'REAL',
#     'td_pct': 'REAL',
#     'sack_pct': 'REAL',
#     'int_pct': 'REAL',
#     'wind': 'REAL',
#     'div_game': 'REAL',
#     'opponent': 'TEXT',
#     'spread_line': 'REAL',
#     'total_line': 'REAL',
#     'outdoors': 'INTEGER',
#     'grass': 'INTEGER',
#     'home_team': 'INTEGER',
#     'pred_total': 'REAL',
#     'opp_total': 'REAL'
# }

# # Write the DataFrame to the SQLite table
# quarterback_df.to_sql(table_name, conn, if_exists='replace', index=False, dtype=dtype)

# # Confirm that the data has been written
# print(f"Data written to table {table_name} in SQLite database nfl_dfs.db")

# # Close the connection
# conn.close()
